In [1]:
import pandas as pd
import numpy as np

In [2]:
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [4]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [5]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# Title Basics

In [11]:
basics.replace({'\\N':np.nan}, inplace=True)


In [13]:
basics = basics.dropna(subset=['runtimeMinutes'])

In [14]:
basics['runtimeMinutes'].isna().sum()

0

In [15]:
basics = basics.dropna(subset=['genres'])

In [16]:
basics['genres'].isna().sum()

0

In [17]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          165204
endYear           2774441
runtimeMinutes          0
genres                  0
dtype: int64

In [18]:
basics['startYear'] = basics['startYear'].astype(float)

In [19]:
basics= basics.dropna(subset=['startYear'])

In [20]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2659049 entries, 0 to 9842463
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         object 
 5   startYear       float64
 6   endYear         object 
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(1), object(8)
memory usage: 202.9+ MB


In [21]:
basics_moviefilter = basics['titleType'] == 'movie' 

In [22]:
len(basics_moviefilter)

2659049

In [23]:
year_filter_2000 = basics['startYear'] > 1999

In [24]:
len(year_filter_2000)

2659049

In [25]:
year_filter_2022 = basics['startYear'] < 2023

In [26]:
len(year_filter_2022)

2659049

In [27]:
basics_combined_filter = year_filter_2000 & year_filter_2022 & basics_moviefilter

In [28]:
len(basics_combined_filter)

2659049

In [29]:
basics_df =basics.loc[basics_combined_filter]

In [30]:
is_documentary = basics_df['genres'].str.contains('documentary',case=False)
basics_df = basics_df[~is_documentary]



In [31]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [32]:
keepers = basics_df['tconst'].isin(akas_df['titleId'])
keepers

34803       True
61116       True
67669       True
86801       True
93938       True
           ...  
9842137     True
9842146     True
9842185    False
9842230     True
9842314    False
Name: tconst, Length: 147482, dtype: bool

In [33]:
basics_df = basics_df[keepers]

In [34]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86688 entries, 34803 to 9842230
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86688 non-null  object 
 1   titleType       86688 non-null  object 
 2   primaryTitle    86688 non-null  object 
 3   originalTitle   86688 non-null  object 
 4   isAdult         86688 non-null  object 
 5   startYear       86688 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86688 non-null  object 
 8   genres          86688 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


# AKAS 

In [6]:
akas.replace({'\\N':np.nan}, inplace=True)

In [7]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [8]:
akas_USfilter = akas['region'] == 'US' 

In [9]:
len(akas_USfilter)

35873523

In [10]:
akas_df = akas[akas_USfilter]

# Ratings

In [12]:
ratings.replace({'\\N':np.nan}, inplace=True)

In [35]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1971
1,tt0000002,5.8,263
2,tt0000003,6.5,1816
3,tt0000004,5.6,178
4,tt0000005,6.2,2612


In [36]:
keepers2 = ratings['tconst'].isin(akas_df['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1309290    False
1309291    False
1309292    False
1309293    False
1309294    False
Name: tconst, Length: 1309295, dtype: bool

In [37]:
ratings_df = ratings[keepers2]

# Data Info of each dataframe

In [38]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86688 entries, 34803 to 9842230
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86688 non-null  object 
 1   titleType       86688 non-null  object 
 2   primaryTitle    86688 non-null  object 
 3   originalTitle   86688 non-null  object 
 4   isAdult         86688 non-null  object 
 5   startYear       86688 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86688 non-null  object 
 8   genres          86688 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


In [39]:
akas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1436149 entries, 5 to 35873267
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1436149 non-null  object
 1   ordering         1436149 non-null  int64 
 2   title            1436149 non-null  object
 3   region           1436149 non-null  object
 4   language         3922 non-null     object
 5   types            978749 non-null   object
 6   attributes       46546 non-null    object
 7   isOriginalTitle  1434804 non-null  object
dtypes: int64(1), object(7)
memory usage: 98.6+ MB


In [40]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498034 entries, 0 to 1309270
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         498034 non-null  object 
 1   averageRating  498034 non-null  float64
 2   numVotes       498034 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.2+ MB


In [41]:
import os

In [42]:
os.makedirs('Data/', exist_ok=True)

In [43]:
os.listdir('Data/')

[]

In [44]:
basics_df.to_csv('Data/title_basics.csv.gz', compression='gzip', index=False)

In [45]:
akas_df.to_csv('Data/AKAS.csv.gz', compression='gzip', index=False)

In [46]:
ratings_df.to_csv('Data/ratings.csv.gz', compression='gzip', index=False)